# Customer segmentation using embeddings and traditional ML from

## Setup

In [ ]:
! pip install -r requirements.txt -q

In [13]:
import pandas as pd
import os
import time

from tqdm.notebook import tqdm

from google.cloud import bigquery
from google import genai

from utils.gen_ai_utils import deploy_text_embedding_model

In [ ]:
PROJECT_ID = "TO_DO_DEVELOPER"
GCP_LOCATION = "TO_DO_DEVELOPER"
DATASET_ID = "TO_DO_DEVELOPER"
USER = "TO_DO_DEVELOPER"

GENERATE_SIGNALS = True
GENERATE_DESCRIPTIONS = True
GENERATE_EMBEDDINGS = True
GENERATE_CLUSTERS = True

In [15]:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = GCP_LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"

## Generation of segmentation signals

In [16]:
customer_engagement_signals_sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.customer_engagement_signals` AS
SELECT 
  c.customer_id,
  -- Session engagement metrics
  COUNT(DISTINCT a.session_id) AS total_sessions,
  AVG(a.session_length_minutes) AS avg_session_length,
  MAX(a.session_length_minutes) AS max_session_length,
  SUM(a.hits) AS total_hits,
  AVG(a.hits) AS avg_hits_per_session,
  
  -- Response and interaction metrics
  SUM(CASE WHEN a.response = TRUE THEN 1 ELSE 0 END) AS total_responses,
  SUM(CASE WHEN a.response = TRUE THEN 1 ELSE 0 END) / COUNT(DISTINCT a.session_id) AS response_rate,
  SUM(CASE WHEN a.visited_simulation = TRUE THEN 1 ELSE 0 END) AS simulation_visits,
  
  -- Email marketing effectiveness
  SUM(CASE WHEN a.from_email_marketing = TRUE THEN 1 ELSE 0 END) AS email_driven_sessions,
  SUM(CASE WHEN a.from_email_marketing = TRUE AND a.response = TRUE THEN 1 ELSE 0 END) AS email_responses,
  
  -- Recency metrics
  DATE_DIFF(CURRENT_DATE(), MAX(CAST(a.session_start AS DATE)), DAY) AS days_since_last_session
FROM 
  `{PROJECT_ID}.{DATASET_ID}.customers` c
LEFT JOIN 
  `{PROJECT_ID}.{DATASET_ID}.analytics` a ON c.customer_id = a.customer_id
GROUP BY 
  c.customer_id;"""


customer_risk_signals_sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.customer_risk_signals` AS
SELECT 
  c.customer_id,
  -- Risk metrics
  p.risk_profile,
  p.num_accidents,
  p.years_with_license,
  p.years_with_license / NULLIF(c.age - 18, 0) AS license_age_ratio,
  
  -- Vehicle and coverage details
  p.car_brand,
  p.car_year,
  EXTRACT(YEAR FROM CURRENT_DATE()) - p.car_year AS vehicle_age,
  p.has_garage,
  p.has_second_driver,
  
  -- Policy value metrics
  p.premium_amount,
  p.coverage_level,
  p.payment_frequency,
  
  -- Policy tenure
  DATE_DIFF(CURRENT_DATE(), CAST(p.start_date AS DATE), DAY) AS policy_tenure_days
FROM 
  `{PROJECT_ID}.{DATASET_ID}.customers` c
LEFT JOIN 
  `{PROJECT_ID}.{DATASET_ID}.policies` p ON c.customer_id = p.customer_id;"""


customer_device_signals_sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.customer_device_signals` AS
SELECT 
  c.customer_id,
  -- Device usage metrics
  COUNT(DISTINCT a.session_id) AS total_sessions,
  SUM(CASE WHEN a.device_type = 'Mobile' THEN 1 ELSE 0 END) AS mobile_sessions,
  SUM(CASE WHEN a.device_type = 'Desktop' THEN 1 ELSE 0 END) AS desktop_sessions,
  SUM(CASE WHEN a.device_type = 'Tablet' THEN 1 ELSE 0 END) AS tablet_sessions,
  
  -- Device preference ratios
  SAFE_DIVIDE(SUM(CASE WHEN a.device_type = 'Mobile' THEN 1 ELSE 0 END), COUNT(DISTINCT a.session_id)) AS mobile_ratio,
  SAFE_DIVIDE(SUM(CASE WHEN a.device_type = 'Desktop' THEN 1 ELSE 0 END), COUNT(DISTINCT a.session_id)) AS desktop_ratio,
  SAFE_DIVIDE(SUM(CASE WHEN a.device_type = 'Tablet' THEN 1 ELSE 0 END), COUNT(DISTINCT a.session_id)) AS tablet_ratio,
  
  -- Browser preferences
  SUM(CASE WHEN a.browser = 'Chrome' THEN 1 ELSE 0 END) AS chrome_sessions,
  SUM(CASE WHEN a.browser = 'Firefox' THEN 1 ELSE 0 END) AS firefox_sessions,
  SUM(CASE WHEN a.browser = 'Safari' THEN 1 ELSE 0 END) AS safari_sessions,
  SUM(CASE WHEN a.browser = 'Edge' THEN 1 ELSE 0 END) AS edge_sessions,
  
  -- Operating system preferences
  SUM(CASE WHEN a.operating_system = 'Windows' THEN 1 ELSE 0 END) AS windows_sessions,
  SUM(CASE WHEN a.operating_system = 'MacOS' THEN 1 ELSE 0 END) AS macos_sessions,
  SUM(CASE WHEN a.operating_system = 'iOS' THEN 1 ELSE 0 END) AS ios_sessions,
  SUM(CASE WHEN a.operating_system = 'Android' THEN 1 ELSE 0 END) AS android_sessions
FROM 
  `{PROJECT_ID}.{DATASET_ID}.customers` c
LEFT JOIN 
  `{PROJECT_ID}.{DATASET_ID}.analytics` a ON c.customer_id = a.customer_id
GROUP BY 
  c.customer_id;"""


customer_lifecycle_signals_sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.customer_lifecycle_signals` AS
SELECT 
  c.customer_id,
  -- Demographic signals
  c.age,
  c.gender,
  c.country,
  c.state,
  
  -- Lifecycle metrics
  DATE_DIFF(CURRENT_DATE(), CAST(c.registration_date AS DATE), DAY) AS customer_tenure_days,
  DATE_DIFF(CURRENT_DATE(), CAST(p.start_date AS DATE), DAY) AS policy_tenure_days,
  DATE_DIFF(CAST(p.start_date AS DATE), CAST(c.registration_date AS DATE), DAY) AS days_to_purchase,
  
  -- Derived lifecycle metrics
  CASE WHEN p.start_date IS NOT NULL THEN 1 ELSE 0 END AS is_converted,
  EXTRACT(YEAR FROM CURRENT_DATE()) - EXTRACT(YEAR FROM CAST(c.birth_date AS DATE)) AS calculated_age,
  
  -- Location-based signals
  c.postal_code,
  c.city
FROM 
  `{PROJECT_ID}.{DATASET_ID}.customers` c
LEFT JOIN 
  `{PROJECT_ID}.{DATASET_ID}.policies` p ON c.customer_id = p.customer_id;"""


customer_segmentation_signals_sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.customer_segmentation_signals` AS
SELECT 
  c.customer_id,
  
  -- Basic customer information
  c.age,
  c.gender,
  
  -- Engagement signals
  eng.total_sessions,
  eng.avg_session_length,
  eng.total_hits,
  eng.total_responses,
  eng.response_rate,
  eng.simulation_visits,
  eng.email_driven_sessions,
  eng.days_since_last_session,
  
  -- Risk and policy signals
  risk.risk_profile,
  risk.num_accidents,
  risk.years_with_license,
  risk.license_age_ratio,
  risk.vehicle_age,
  risk.has_garage,
  risk.has_second_driver,
  risk.premium_amount,
  risk.coverage_level,
  risk.policy_tenure_days,
  
  -- Device preference signals
  dev.mobile_ratio,
  dev.desktop_ratio,
  dev.tablet_ratio,
  
  -- Lifecycle signals
  life.customer_tenure_days,
  life.days_to_purchase,
  
  -- Derived high-value signals
  (eng.total_sessions * eng.avg_session_length) AS total_engagement_time,
  (risk.premium_amount / NULLIF(risk.num_accidents + 1, 0)) AS premium_risk_ratio,
  (eng.simulation_visits / NULLIF(eng.total_sessions, 0)) AS simulation_visit_rate,
  (eng.email_responses / NULLIF(eng.email_driven_sessions, 0)) AS email_effectiveness,
  CASE WHEN risk.policy_tenure_days > 180 AND eng.total_sessions > 5 THEN 1 ELSE 0 END AS is_engaged_customer,
  CASE WHEN risk.premium_amount > 1000 THEN 1 ELSE 0 END AS is_high_value
FROM 
  `{PROJECT_ID}.{DATASET_ID}.customers` c
LEFT JOIN 
  `{PROJECT_ID}.{DATASET_ID}.customer_engagement_signals` eng ON c.customer_id = eng.customer_id
LEFT JOIN 
  `{PROJECT_ID}.{DATASET_ID}.customer_risk_signals` risk ON c.customer_id = risk.customer_id
LEFT JOIN 
  `{PROJECT_ID}.{DATASET_ID}.customer_device_signals` dev ON c.customer_id = dev.customer_id
LEFT JOIN 
  `{PROJECT_ID}.{DATASET_ID}.customer_lifecycle_signals` life ON c.customer_id = life.customer_id;"""

In [17]:
if GENERATE_SIGNALS:
    client = bigquery.Client()
    client.query_and_wait(customer_engagement_signals_sql)
    client.query_and_wait(customer_risk_signals_sql)
    client.query_and_wait(customer_device_signals_sql)
    client.query_and_wait(customer_lifecycle_signals_sql)
    client.query_and_wait(customer_segmentation_signals_sql)


## Approach 1 - Using semantic embeddings

In [18]:
def generate_customer_description(row):
    customer_data = {
        "demographics": {
            "age": row["age"],
            "gender": row["gender"],
            "tenure_months": round(row["customer_tenure_days"]/30.0, 1)
        },
        "policy": {
            "coverage_level": row["coverage_level"],
            "premium_amount": round(row["premium_amount"], 2),
            "risk_profile": row["risk_profile"],
            "has_second_driver": row["has_second_driver"],
            "has_garage": row["has_garage"]
        },
        "driving": {
            "license_years": row["years_with_license"], 
            "accidents": row["num_accidents"],
            "vehicle_age": row["vehicle_age"]
        },
        "engagement": {
            "total_sessions": row["total_sessions"],
            "avg_session_length": round(row["avg_session_length"], 1),
            "days_since_last": row["days_since_last_session"],
            "simulation_visits": row["simulation_visits"],
            "total_responses": row["total_responses"],
            "response_rate": row["response_rate"],
            "email_sessions": row["email_driven_sessions"]
        },
        "devices": {
            "mobile_ratio": row["mobile_ratio"],
            "desktop_ratio": row["desktop_ratio"], 
            "tablet_ratio": row["tablet_ratio"]
        },
        "indicators": {
            "days_to_purchase": row["days_to_purchase"],
            "is_high_value": row["is_high_value"],
            "is_engaged": row["is_engaged_customer"]
        }
    }

    # Prompt for Gemini
    prompt = f"""
    Generate a natural, detailed description of an insurance customer based on the following data:
    {customer_data}
    
    The description should cover:
    - Demographics and customer tenure
    - Policy details and risk profile  
    - Driving history and vehicle details
    - Digital engagement patterns
    - Device preferences
    - Purchase behavior
    - Value and engagement indicators
    
    Write in a professional but conversational tone.
    """

    # Call Gemini API
    client = genai.Client(project=PROJECT_ID,location='us-central1', vertexai=True
    response = client.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=prompt,
    )

    if response.text:
        return response.text
    else:
        # Fallback to structured description if Gemini fails
        device_pref = get_device_preference(row["mobile_ratio"], row["desktop_ratio"], row["tablet_ratio"])

        description = [
            f"This is a {row['age']}-year-old {row['gender'].lower()} customer who has been with us for {round(row['customer_tenure_days']/30.0, 1)} months.",
            f"They have a {row['coverage_level'].lower()} coverage policy (${round(row['premium_amount'], 2)}) and are considered {row['risk_profile'].lower()} risk.",
            f"With {row['years_with_license']} years of driving experience and {row['num_accidents']} accident(s), they drive a {row['vehicle_age']}-year-old vehicle.",
            f"Their digital engagement shows {row['total_sessions']} site visits averaging {round(row['avg_session_length'], 1)} minutes, primarily using {device_pref}.",
            f"{'A high-value' if row['is_high_value'] == 1 else 'Not currently a high-value'} customer with {'strong' if row['is_engaged_customer'] == 1 else 'limited'} platform engagement."
        ]

        return " ".join(description)

def get_device_preference(mobile_ratio, desktop_ratio, tablet_ratio):
    """Helper function to determine device preference"""
    if mobile_ratio > 0.5:
        return "mobile devices"
    elif desktop_ratio > 0.5:
        return "desktop computers"
    elif tablet_ratio > 0.5:
        return "tablet devices"
    else:
        return "multiple device types"


def generate_descriptions_from_bigquery(project_id=None, dataset_id=None):
    """
    Load customer segmentation signals from BigQuery and generate descriptions using pandas.

    Args:
        project_id (str, optional): Google Cloud project ID
        dataset_id (str, optional): BigQuery dataset ID

    Returns:
        pandas.DataFrame: DataFrame with customer_id and customer_description
    """
    # Load data from BigQuery
    client = bigquery.Client(project=project_id)
    query = f"""
    SELECT * FROM `{project_id}.{dataset_id}.customer_segmentation_signals`
    """
    signals_df = client.query(query).to_dataframe()

    # Generate descriptions with progress bar and save checkpoints
    batch_size = 100
    total_batches = len(signals_df) // batch_size + (1 if len(signals_df) % batch_size != 0 else 0)
    
    descriptions_df = pd.DataFrame(columns=["customer_id", "customer_description"])
    
    for batch_num in range(total_batches):
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(signals_df))
        batch_df = signals_df.iloc[start_idx:end_idx]
        
        print(f"Processing batch {batch_num + 1}/{total_batches}")
        
        tqdm.pandas(desc=f"Generating descriptions for batch {batch_num + 1}")
        batch_df["customer_description"] = batch_df.progress_apply(
            generate_customer_description, axis=1
        )
        
        # Add batch results to final dataframe
        batch_descriptions = batch_df[["customer_id", "customer_description"]]
        descriptions_df = pd.concat([descriptions_df, batch_descriptions])
        
        # Save checkpoint to CSV
        checkpoint_file = f"customer_descriptions_checkpoint_{batch_num}.csv"
        descriptions_df.to_csv(checkpoint_file, index=False)
        print(f"Saved checkpoint to {checkpoint_file}")

    # Save final results to BigQuery
    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    )
    table_id = f"{project_id}.{dataset_id}.customer_descriptions"
    job = client.load_table_from_dataframe(
        descriptions_df, table_id, job_config=job_config
    )
    job.result()
    print(f"Saved all descriptions to BigQuery table: {table_id}")

    return descriptions_df

In [19]:
if GENERATE_DESCRIPTIONS:
    generate_descriptions_from_bigquery(PROJECT_ID, DATASET_ID)

In [20]:

params={
        "project_id": PROJECT_ID,
        "bigquery_location": str.lower(GCP_LOCATION),
        "dataset_id": DATASET_ID,
        "user": USER,
        "vertex_ai_connection_name": "vertex-ai-conn",
    }

deploy_text_embedding_model(params)


Checking existing connections...
Found connection: projects/1063524325524/locations/us/connections/ai_lakehouse
Found connection: projects/1063524325524/locations/us/connections/bigframes-default-connection
Found connection: projects/1063524325524/locations/us/connections/biglake-conn
Found connection: projects/1063524325524/locations/us/connections/biglake-connection
Found connection: projects/1063524325524/locations/us/connections/biglake-managed
Found connection: projects/1063524325524/locations/us/connections/biglake_vertexai_connection
Found connection: projects/1063524325524/locations/us/connections/bqframes
Found connection: projects/1063524325524/locations/us/connections/caixabank-genai
Found connection: projects/1063524325524/locations/us/connections/genai
Found connection: projects/1063524325524/locations/us/connections/next_demo
Found connection: projects/1063524325524/locations/us/connections/spark-sp
Found connection: projects/1063524325524/locations/us/connections/vertex-

In [21]:
if GENERATE_EMBEDDINGS:
    columns_to_embed = [
        ["customer_description", "SEMANTIC_SIMILARITY"],
    ]
    client = bigquery.Client()
    for column in columns_to_embed:
        print(
            f"Populating {column[0]}_embedding column with embeddings using task type {column[1]}"
        )
    
        sleep_time_seconds = 5
        sql = f"""CREATE OR REPLACE TABLE 
                `{PROJECT_ID}.{DATASET_ID}.customer_description_embeddings` AS
                SELECT customer_id, ml_generate_embedding_result as {column[0]}_embedding
                FROM ML.GENERATE_EMBEDDING(
                    MODEL `{PROJECT_ID}.{DATASET_ID}.google-textembedding`,
                    (SELECT customer_id, customer_description as content
                    FROM `{PROJECT_ID}.{DATASET_ID}.customer_descriptions`),
                    STRUCT(
                      TRUE AS flatten_json_output,
                      '{column[1]}' as task_type,
                      768 AS output_dimensionality
                      )
                ) """
        client.query_and_wait(sql)
        time.sleep(5)

Populating customer_description_embedding column with embeddings using task type SEMANTIC_SIMILARITY


In [29]:
USER_QUERY = "young male driver with a Standard coverage hihgh-value customer, but with low engagement with digital channels"
sql = f"""
WITH customer_ids AS (
SELECT
  distance,
  base.customer_id,
FROM VECTOR_SEARCH(
  -- base table or subquery
  (
    SELECT * FROM `{DATASET_ID}.customer_description_embeddings`
  ),

  -- embedding column to search in base table - must be of type ARRAY
  'customer_description_embedding',

  -- query table or subquery - this is where you generate the search embedding
  (
    SELECT ml_generate_embedding_result, content AS query
    FROM ML.GENERATE_EMBEDDING(
      MODEL `{DATASET_ID}.google-textembedding`,
        (
          -- Search term
          SELECT "{USER_QUERY}" AS content
        ),
        STRUCT(
          TRUE AS flatten_json_output,
          'SEMANTIC_SIMILARITY' as task_type,
          768 AS output_dimensionality
        )
    )
  ),
  top_k => 5,
  distance_type => 'COSINE'
))
SELECT * FROM `insurance_data.customers` c
JOIN `{DATASET_ID}.policies` p
ON  c.customer_id = p.customer_id
WHERE c.customer_id IN (SELECT customer_id FROM customer_ids)
"""

client = bigquery.Client()
df = client.query_and_wait(sql).to_dataframe()
df.head()

,customer_id,first_name,last_name,gender,age,birth_date,email,phone_number,street_address,city,...,car_model,car_year,has_garage,has_second_driver,years_with_license,num_accidents,risk_profile,premium_amount,coverage_level,payment_frequency
0,eeaca1c2-6065-44b4-8ba1-56aff193d541,Bruce,Nolan,Male,18,2005-02-22,angelarivera@example.org,955-838-8287x204,6608 Amber Park Suite 587,Lake Elizabethton,...,Leave,2021,True,False,0,0,Medium,1261.124753,Standard,Monthly
1,94efd608-4485-4a60-9106-8da2bb3d6f9a,Curtis,Gonzalez,Male,23,1945-03-14,antonio83@example.org,+1-612-272-4901,3934 Mccarty Path Suite 738,Johnmouth,...,Meeting,2015,False,False,5,0,Medium,772.681853,Basic,Semi-Annual
2,b1f62c86-7178-4d44-909b-34569e138ed6,Jason,Pearson,Male,31,1944-01-09,brittanybishop@example.com,557.384.5907,211 Ian Inlet,North Teresa,...,Director,2000,True,False,2,0,Medium,1697.766762,Standard,Semi-Annual
3,d260cd3a-b8a3-4ab5-94b2-1f22d8eee9a3,Joshua,Parsons,Male,23,1964-04-29,benjamin34@example.org,890-985-7346,897 Christopher Isle Suite 227,New Susanstad,...,Administration,2002,False,False,5,2,Medium,1410.120928,Standard,Monthly
4,31bd5d42-46b8-44fb-8524-31a2fdad075d,Michael,Lynch,Male,23,1975-11-23,atorres@example.com,(493)371-7847,1727 Nichols Valleys Suite 529,South Sean,...,Entire,2015,True,False,5,0,High,1636.606414,Basic,Annual


## Aproach 2 - Using traditional unsupervised learning (k-means)

In [32]:
sql = f"""CREATE OR REPLACE MODEL `{DATASET_ID}.customer_clusters_kmeans`
OPTIONS (
  model_type = 'kmeans',
  num_clusters = 8)
AS
SELECT * EXCEPT(customer_id) FROM `{DATASET_ID}.customer_segmentation_signals`;"""
client = bigquery.Client()
df = client.query_and_wait(sql)

In [ ]:
sql = f"""SELECT
  *
FROM
  ML.CENTROIDS(MODEL `{DATASET_ID}.customer_clusters_kmeans`)"""
client = bigquery.Client()
df = client.query_and_wait(sql).to_dataframe()
df.head()

,centroid_id,feature,numerical_value,categorical_value
0,1,age,6.148148e+01,[]
1,1,gender,NaN,"[{'category': 'Female', 'value': 0.48888888888..."
2,1,total_sessions,2.903704e+00,[]
3,1,avg_session_length,3.117049e+01,[]
4,1,total_hits,6.787037e+01,[]
...,...,...,...,...
243,8,premium_risk_ratio,5.123056e+02,[]
244,8,simulation_visit_rate,3.032796e-01,[]
245,8,email_effectiveness,2.180780e-01,[]
246,8,is_engaged_customer,-1.110223e-16,[]


In [40]:
sql = f"""CREATE OR REPLACE TABLE `{DATASET_ID}.customer_clusters_predictions` AS
SELECT *
EXCEPT (nearest_centroids_distance)
FROM
ML.PREDICT(
  MODEL `{DATASET_ID}.customer_clusters_kmeans`,
  (
    SELECT *
    FROM
      `{DATASET_ID}.customer_segmentation_signals`
  ));"""
client = bigquery.Client()
df = client.query_and_wait(sql)

In [42]:
sql = f"""WITH five_customer_per_cluster AS (
  SELECT 
    cp.customer_id, 
    cp.centroid_id AS cluster_id,
    ROW_NUMBER() OVER (PARTITION BY cp.centroid_id ORDER BY RAND()) AS rn
  FROM `{DATASET_ID}.customer_clusters_predictions` cp
)

SELECT 
  *
FROM five_customer_per_cluster cc
JOIN `{DATASET_ID}.customers` c ON cc.customer_id = c.customer_id
JOIN `{DATASET_ID}.policies` p ON c.customer_id = p.customer_id
JOIN `{DATASET_ID}.analytics` a ON c.customer_id = a.customer_id
WHERE cc.rn < 6
ORDER BY cc.cluster_id"""
client = bigquery.Client()
df = client.query_and_wait(sql).to_dataframe()
df.head()

,customer_id,cluster_id,rn,customer_id_1,first_name,last_name,gender,age,birth_date,email,...,session_end,session_length_minutes,pages_visited,visited_simulation,from_email_marketing,hits,response,device_type,browser,operating_system
0,7bcfccfb-1388-425e-850f-f0d9dbff74e2,1,1,7bcfccfb-1388-425e-850f-f0d9dbff74e2,John,Reese,Male,46,1965-08-28,robinneal@example.com,...,2025-02-21 07:56:38.858415,34,2,False,False,4,False,Desktop,Safari,Linux
1,7bcfccfb-1388-425e-850f-f0d9dbff74e2,1,1,7bcfccfb-1388-425e-850f-f0d9dbff74e2,John,Reese,Male,46,1965-08-28,robinneal@example.com,...,2025-02-18 23:29:22.340606,12,10,False,True,10,False,Tablet,Safari,Linux
2,44a9ee10-5687-4026-b22a-9431f4b50bbb,1,2,44a9ee10-5687-4026-b22a-9431f4b50bbb,Erica,Fowler,Female,78,1966-11-11,wwatson@example.com,...,2025-03-04 16:21:45.460274,13,5,False,False,15,False,Mobile,Safari,Android
3,44a9ee10-5687-4026-b22a-9431f4b50bbb,1,2,44a9ee10-5687-4026-b22a-9431f4b50bbb,Erica,Fowler,Female,78,1966-11-11,wwatson@example.com,...,2025-02-22 00:17:33.372143,51,7,True,True,7,False,Desktop,Edge,MacOS
4,44a9ee10-5687-4026-b22a-9431f4b50bbb,1,2,44a9ee10-5687-4026-b22a-9431f4b50bbb,Erica,Fowler,Female,78,1966-11-11,wwatson@example.com,...,2025-02-21 01:16:19.392124,16,7,False,False,14,False,Desktop,Safari,Windows


In [51]:
clusters = {}
for cluster_id, cluster_df in df.groupby("cluster_id"):
    
    stats = {
        "avg_age": cluster_df["age"].mean(),
        "avg_premium": cluster_df["premium_amount"].mean(),
        "gender_distribution": cluster_df["gender"].value_counts().to_dict(),
        "risk_profile": cluster_df["risk_profile"].value_counts().to_dict(),
        "coverage_levels": cluster_df["coverage_level"].value_counts().to_dict(),
        "payment_frequency": cluster_df["payment_frequency"]
        .value_counts()
        .to_dict(),
        "avg_years_license": cluster_df["years_with_license"].mean(),
        "avg_vehicle_age": (
            2025 - cluster_df["car_year"]
        ).mean(),  # Approximating current year
        "accident_rate": cluster_df["num_accidents"].mean(),
        "garage_percentage": (cluster_df["has_garage"].sum() / len(cluster_df))
        * 100,
        "second_driver_percentage": (
            cluster_df["has_second_driver"].sum() / len(cluster_df)
        )
        * 100,
        # Analytics stats
        "avg_session_length": (
            cluster_df["session_length_minutes"].mean()
            if "session_length_minutes" in cluster_df.columns
            else "N/A"
        ),
        "avg_hits": (
            cluster_df["hits"].mean() if "hits" in cluster_df.columns else "N/A"
        ),
        "simulation_visits": (
            (cluster_df["visited_simulation"].sum() / len(cluster_df)) * 100
            if "visited_simulation" in cluster_df.columns
            else "N/A"
        ),
    }

    clusters[f"Cluster {cluster_id}"] = {
        "samples": cluster_df.iloc[:3].to_dict(
            "records"
        ), 
        "stats": stats,
    }

# Create a prompt that describes all clusters
prompt = """
You are an experienced insurance marketing analyst specializing in customer segmentation for a car insurance company.
Below is data from all customer segments identified through cluster analysis:
"""

for cluster_name, cluster_data in clusters.items():
    prompt += f"\n\n## {cluster_name}\n"
    prompt += f"### Key Statistics:\n"
    stats = cluster_data["stats"]

    # Add statistics that exist
    for stat_name, stat_value in stats.items():
        if stat_value != "N/A":
            # Format based on stat type
            if isinstance(stat_value, float):
                if "percentage" in stat_name:
                    prompt += f"- {stat_name.replace('_', ' ').title()}: {stat_value:.1f}%\n"
                elif "avg" in stat_name:
                    prompt += f"- {stat_name.replace('_', ' ').title()}: {stat_value:.1f}\n"
                else:
                    prompt += f"- {stat_name.replace('_', ' ').title()}: {stat_value:.2f}\n"
            else:
                prompt += f"- {stat_name.replace('_', ' ').title()}: {stat_value}\n"

    # Include simplified sample customer data
    prompt += f"\n### Sample Customers (3 of 5 analyzed):\n"
    samples = cluster_data["samples"]
    for i, sample in enumerate(samples):
        # Create a simplified version of the sample to avoid overwhelming the model
        simple_sample = {
            "age": sample["age"],
            "gender": sample["gender"],
            "risk_profile": sample["risk_profile"],
            "premium_amount": sample["premium_amount"],
            "coverage_level": sample["coverage_level"],
            "years_with_license": sample["years_with_license"],
            "num_accidents": sample["num_accidents"],
            "has_garage": sample["has_garage"],
            "has_second_driver": sample["has_second_driver"],
            "payment_frequency": sample["payment_frequency"],
        }
        prompt += f"Sample {i+1}: {simple_sample}\n"

prompt += """

Based on this comprehensive data, please provide:

1. A concise, meaningful name for each customer segment (e.g., "Safe Young Drivers", "Experienced Family Commuters", "High-Premium Luxury Vehicle Owners")

2. For each segment, a detailed description including:
   - Key demographic characteristics
   - Typical risk factors and insurance needs
   - Policy preferences and price sensitivity
   - Likely life stage and circumstances
   - Key distinguishing features compared to other segments

Your analysis should be insightful, data-driven, and actionable for marketing teams.
"""
print(prompt)




You are an experienced insurance marketing analyst specializing in customer segmentation for a car insurance company.
Below is data from all customer segments identified through cluster analysis:


## Cluster 1
### Key Statistics:
- Avg Age: 62.2
- Avg Premium: 936.9
- Gender Distribution: {'Male': 8, 'Female': 4}
- Risk Profile: {'Medium': 7, 'Low': 5}
- Coverage Levels: {'Standard': 7, 'Basic': 3, 'Premium': 2}
- Payment Frequency: {'Semi-Annual': 4, 'Annual': 4, 'Monthly': 2, 'Quarterly': 2}
- Avg Years License: 32.3
- Avg Vehicle Age: 10.7
- Accident Rate: 0.83
- Garage Percentage: 25.0%
- Second Driver Percentage: 58.3%
- Avg Session Length: 26.5
- Avg Hits: 17.2
- Simulation Visits: 50.00

### Sample Customers (3 of 5 analyzed):
Sample 1: {'age': 46, 'gender': 'Male', 'risk_profile': 'Low', 'premium_amount': 732.6729320289477, 'coverage_level': 'Standard', 'years_with_license': 28, 'num_accidents': 0, 'has_garage': False, 'has_second_driver': True, 'payment_frequency': 'Monthly'

In [61]:
from pydantic import BaseModel
class CustomerSegment(BaseModel):
    cluster_id: int
    segment_name: str
    description: str
    keywords: list[str]


client = genai.Client(project=PROJECT_ID,location='us-central1', vertexai=True)
response = client.models.generate_content(
    model="gemini-2.0-pro-exp-02-05",
    contents=prompt,
    config={
        "response_mime_type": "application/json",
        "response_schema": list[CustomerSegment],
    },
)

if response.text:
    segments: list[CustomerSegment] = response.parsed

In [62]:
segments

[CustomerSegment(cluster_id=1, segment_name='Mature Standard Savers', description='Older, experienced drivers with moderate risk profiles. They tend to favor standard coverage and are less price-sensitive, preferring semi-annual or annual payments. Likely retirees with lower driving needs but higher accident rates than some other segments.', keywords=['Retiree', 'Experienced', 'Standard Coverage', 'Semi-Annual Payment', 'Annual Payment', 'Moderate Risk']),
 CustomerSegment(cluster_id=2, segment_name='Premium Suburban Protectors', description='Middle-aged to older drivers with a mix of risk profiles, but leaning towards lower risk. They show a strong preference for premium coverage, suggesting a desire for comprehensive protection. A higher proportion have garages, indicating potentially more valuable vehicles. Less price sensitive.', keywords=['Middle-Aged', 'Premium Coverage', 'Low Risk', 'Garage', 'Higher Premium']),
 CustomerSegment(cluster_id=3, segment_name='Male Premium Multi-Car